In [5]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys

In [6]:
#Read CSV file containing the persons from the North East of England
df_persons_NE_inactive_20211117__dir = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity' # use your path
df_persons_NE_inactive_20211117_file = os.path.join(df_persons_NE_inactive_20211117__dir, "df_persons_NE_inactive_20211117.csv")
df_persons_NE_inactive_20211117 = pd.read_csv(df_persons_NE_inactive_20211117_file, index_col=None, header=0)


In [3]:
#Read CSV file containing the households from the North East of England
df_households_NE_extended__dir = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019' # use your path
df_households_NE_extended_file = os.path.join(df_households_NE_extended__dir, "df_households_NE_extended.csv")
df_households_NE_extended = pd.read_csv(df_households_NE_extended_file, index_col=None, header=0)


In [4]:
#Read CSV file containing the economic activities per OA level, related to inactive people (QS601EW Economic Activity)
## This dataset has been already cleaned and column names have been updated
## This dataset contains information about the number of inactive people per location (OA area).
df_QS601EW_dir = r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Data\Census\Datasets_downloaded\Economic_activity\Selected' # use your path
df_QS601EW_file = os.path.join(df_QS601EW_dir, "QS601EW_20211104.csv")
df_QS601EW_inactive = pd.read_csv(df_QS601EW_file, index_col=None, header=0)


In [5]:
# Create a dataframe only containing the people that are Economic_activity = 'Inactive' and age between 14 and 64
df_persons_NE_inactive_16_64 = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >= 16) & (df_persons_NE_inactive_20211117['Age'] <= 64)]


In [6]:
# Total number of persons that have to be assinged an inactive category:
len(df_persons_NE_inactive_16_64)

407771

## Start selecting inactive categories (student, looking after home or family, retired, sick or other) based on sociodemographic characteristics of the persons

### Students

In [13]:
## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
gender_list = [1,2]

#Create an empty list where the small blocks of dataframes of STUDENTS will be stored
persons_NE_inactive_students_chosen_all_list = []


# % inactive conversor value 2011 to 2019
## This value is converting the % of inactive people (each type) from 2011 to 2019 based on data from "Regional labour market statistics: HI01 Headline indicators for the North East"
inactive_student_conversor = 1.202


#  Relationship between male and female people per reason of inactivity (male + female =1)
## These values are coming from Regional labour market statistics:HI01 Headline indicators for the North East. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_student = 0.570

# Females
percentage_2_student = 0.430


#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:

    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
    
    
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]

    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2019 = len(df_persons_NE_inactive_16_64_OAarea)

    
    # Calculate the ratio of people 2019 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2019_2011 = total_inactive_in_OA_2019/total_QS601EW_inactive_in_OA_2011

    else:
        ratio_people_2019_2011 = 1

    
    # Get the VALUE of the number of people living in the seleted OA area that are inactive STUDENTS from table QS601EW
    total_QS601EW_inactive_students_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Student')]

   
    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to student ("total_QS601EW_inactive_students_in_OA_2011") 
    # 2. the "inactive_student_conversor" value
    # 3. the ratio of people 2019 vs 2011 ("ratio_people_2019_2011")
    
    # New value for 2019 =  (Value from 2011 (table QS601EW)) * ("inactive_student_conversor"  value) * (ratio of people 2019 vs 2011 ("ratio_people_2019_2011")) 
    ## This value is the number of people that will be assigned "STUDENT" in the OA area selected
    inactive_students_2019 = int(round(total_QS601EW_inactive_students_in_OA_2011 * inactive_student_conversor * ratio_people_2019_2011,0)) 
    

    
    ## START IDENTIFYING THE STUDENTS
    
    for gender in gender_list:
        
        # FIRST SELECTION
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. QS420_CELL = 26  (agents living in buildings related to education)
        ## 2. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously

        # Select those people QS420_CELL = 26 (agents living in buildings related to education --> student accomodations (assumption))
        globals()[f"df_{gender}_QS420_CELL_26"] = df_persons_NE_inactive_16_64_OAarea.loc[(df_persons_NE_inactive_16_64_OAarea['QS420_CELL'] == 26) & (df_persons_NE_inactive_16_64_OAarea['Sex'] == gender)]

        # If there are people NSSEC = 9 (students):
        if len(globals()[f"df_{gender}_QS420_CELL_26"]) > 0:
            
            
            if ((int(round(inactive_students_2019 * globals()[f"percentage_{gender}_student"],0))) < len(globals()[f"df_{gender}_QS420_CELL_26"])):
                
                # Select randomly the number of people that will be assigned as inactive               
                globals()[f"df_{gender}_first_students_chosen"] = globals()[f"df_{gender}_QS420_CELL_26"].sample(int(round(inactive_students_2019 * globals()[f"percentage_{gender}_student"],0)))
                
           
            else:
                
                # Select all people that will be assigned as students
                globals()[f"df_{gender}_first_students_chosen"] = globals()[f"df_{gender}_QS420_CELL_26"]
                
                # Not all students have already been selected, just a few of them.
                # Second selection should be run
            
        # If there is not any student in the OA area:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            globals()[f"df_{gender}_first_students_chosen"] =  pd.DataFrame()
                
            # None students have already been selected.
            # Second selection should be run.
        
        
        #######################################################################################################################
        
        # SECOND SELECTION:
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. NSSEC == 9 (Students)
        ## 2. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously
        
        # Remaining NUMBER of people to be assigned as INACTIVE:
        second_selection_inactive_students_2019 = int(round(inactive_students_2019 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"])
        
        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_plus_first_studentschosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea, globals()[f"df_{gender}_first_students_chosen"]]))
              
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_second_selection = df_persons_NE_inactive_16_64_OAarea_plus_first_studentschosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        # Select those that are students
        globals()[f"df_{gender}_NSSEC_9"] = df_persons_NE_inactive_16_64_OAarea_second_selection.loc[(df_persons_NE_inactive_16_64_OAarea_second_selection['Sex'] == gender) & (df_persons_NE_inactive_16_64_OAarea_second_selection['NSSEC'] == 9)]
        
        
        if (second_selection_inactive_students_2019 > 0):
        
            # If there are people NSSEC = 9 (students):
            if len(globals()[f"df_{gender}_NSSEC_9"]) > 0:
            
                if ((second_selection_inactive_students_2019) < len(globals()[f"df_{gender}_NSSEC_9"])):
                
                    # Select randomly the number of people that will be assigned as inactive               
                    globals()[f"df_{gender}_second_students_chosen"] = globals()[f"df_{gender}_NSSEC_9"].sample(second_selection_inactive_students_2019)
                
                else:
                
                    # Select all people that will be assigned as students
                    globals()[f"df_{gender}_second_students_chosen"] = globals()[f"df_{gender}_NSSEC_9"]
                
                    # Not all students have already been selected, just a few of them.
                    # Second selection should be run
            
            # If there is not any person living in multi-person households and are aged 16-35 in the OA area:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                globals()[f"df_{gender}_second_students_chosen"] =  pd.DataFrame()
                
                # None persons living in multi-person households and are aged 16-35 in the OA area have been selected
                # Third selection should be run.
                
        # if second_selection_inactive_students_2019 = 0, then no more studetns should be selected
        else:
            
            globals()[f"df_{gender}_second_students_chosen"] =  pd.DataFrame()
                    

        ################################################################################################################
        
        # THIRD SELECTION:
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. LC4408_C_AHTHUK11 = 5 (Domestic situation = Multi-person household)
        ## 2. Age: people 16-35 years old
        ## 3. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously

        
        # Remaining NUMBER of people to be assigned as INACTIVE:
        third_selection_inactive_students_2019 = int(round(inactive_students_2019 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"]) - len(globals()[f"df_{gender}_second_students_chosen"])
        
        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_second_selection_plus_second_studentschosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea_second_selection, globals()[f"df_{gender}_second_students_chosen"]]))
        
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_third_selection = df_persons_NE_inactive_16_64_OAarea_second_selection_plus_second_studentschosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        # Select those that LC4408_C_AHTHUK11 = 5 and aged 16-35 (specific gender and OA area)
        globals()[f"df_{gender}_16_35_AHTHUK11_5"] = df_persons_NE_inactive_16_64_OAarea_third_selection.loc[(df_persons_NE_inactive_16_64_OAarea_third_selection['Sex'] == gender) & (df_persons_NE_inactive_16_64_OAarea_third_selection['Age'] >= 16) & (df_persons_NE_inactive_16_64_OAarea_third_selection['Age'] <= 35) & (df_persons_NE_inactive_16_64_OAarea_third_selection['LC4408_C_AHTHUK11_x'] == 5)]
        

        if (third_selection_inactive_students_2019 > 0):            
                    
            # If there are people LC4408_C_AHTHUK11 = 1,2, OR 3, and aged 16-35 (specific gender and OA area):
            if len(globals()[f"df_{gender}_16_35_AHTHUK11_5"]) > 0:
            
                if (third_selection_inactive_students_2019) < len(globals()[f"df_{gender}_16_35_AHTHUK11_5"]):
                
                    # Select randomly the number of people that will be assigned as inactive               
                    globals()[f"df_{gender}_third_students_chosen"] = globals()[f"df_{gender}_16_35_AHTHUK11_5"].sort_values('Age', ascending = True).head(third_selection_inactive_students_2019)
                
                else:
                
                    # Select all people that will be assigned as students
                    globals()[f"df_{gender}_third_students_chosen"] = globals()[f"df_{gender}_16_35_AHTHUK11_5"]
                
                    # Not all students have already been selected, just a few of them.
                    # Second selection should be run
            
            # If there is not any person living in multi-person households and are aged 16-35 in the OA area:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                globals()[f"df_{gender}_third_students_chosen"] =  pd.DataFrame()
                
                # None persons LC4408_C_AHTHUK11 = 1,2, OR 3, and aged 16-35 (specific gender and OA area) have been selected
                # Forth selection should be run.
        
        # if third_selection_inactive_students_2019 = 0, then no more studetns should be selected
        else:
            
            globals()[f"df_{gender}_third_students_chosen"] =  pd.DataFrame()  
        

        #############################################################################################################
        
        # FORTH SELECTION:
        # SELECT THE STUDENTS BASED ON OTHER SOCIOECONOMIC CHARACTERISTICS:
        ## 1. LC4408_C_AHTHUK11 = 1,2, OR 3 (Domestic situation = one person household, married, cohabitating couple)
        ## 2. Age: people 16-35 years old
        ## 3. Sex: proportions based on variables "percentage_male_student" and "percentage_male_student" assigned previously

        # Remaining NUMBER of people to be assigned as INACTIVE:
        forth_selection_inactive_students_2019 = int(round(inactive_students_2019 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"]) - len(globals()[f"df_{gender}_second_students_chosen"]) -len(globals()[f"df_{gender}_third_students_chosen"])
        
        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_third_selection_plus_third_studentschosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea_third_selection, globals()[f"df_{gender}_third_students_chosen"]]))
        
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_forth_selection = df_persons_NE_inactive_16_64_OAarea_third_selection_plus_third_studentschosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        
        # Select those that LC4408_C_AHTHUK11 = 1,2, OR 3, and aged 16-35 (specific gender and OA area)
        domestic_array = [1,2,3]
        globals()[f"df_{gender}_16_35_domestic123"] = df_persons_NE_inactive_16_64_OAarea_forth_selection.loc[(df_persons_NE_inactive_16_64_OAarea_forth_selection['Sex'] == gender)  & (df_persons_NE_inactive_16_64_OAarea_forth_selection['LC4408_C_AHTHUK11_x'].isin(domestic_array)) & (df_persons_NE_inactive_16_64_OAarea_forth_selection['Age'] >= 16) & (df_persons_NE_inactive_16_64_OAarea_forth_selection['Age'] <= 35)]
        
        
        if (forth_selection_inactive_students_2019 > 0):
                
            # If there are people LC4408_C_AHTHUK11 = 1,2,3 OR 5 :
            if len(globals()[f"df_{gender}_16_35_domestic123"]) > 0:
            
                if ((forth_selection_inactive_students_2019) <= len(globals()[f"df_{gender}_16_35_domestic123"])):
                
                    # Select randomly the number of people that will be assigned as inactive               
                    globals()[f"df_{gender}_forth_students_chosen"] = globals()[f"df_{gender}_16_35_domestic123"].sort_values('Age', ascending = True).head(forth_selection_inactive_students_2019)
           
                else:
                
                    # Select all people that will be assigned as students
                    globals()[f"df_{gender}_forth_students_chosen"] = globals()[f"df_{gender}_16_35_domestic123"]
                
                    # Not all students have already been selected, just a few of them.
                    # Second selection should be run
            
            #If the dataframe does not have any rows, then it will be empty.        
            else:
                globals()[f"df_{gender}_forth_students_chosen"] = pd.DataFrame()
        
        
        # if forth_selection_inactive_students_2019 = 0, then no more studetns should be selected
        else:
             globals()[f"df_{gender}_forth_students_chosen"] = pd.DataFrame()       
        
        
        ###################################################################################################################
        
        ## FIFTH SELECTION
        # Finally, select randomly the remaining ones (leftovers) based on the age only (sort age ascending order and selecte the remaining ones)
        
        # Remaining NUMBER of people to be assigned as INACTIVE:
        fifth_selection_inactive_students_2019 = int(round(inactive_students_2019 * globals()[f"percentage_{gender}_student"],0)) - len(globals()[f"df_{gender}_first_students_chosen"]) - len(globals()[f"df_{gender}_second_students_chosen"]) - len(globals()[f"df_{gender}_third_students_chosen"]) - len(globals()[f"df_{gender}_forth_students_chosen"])

        # Concatenate 
        df_persons_NE_inactive_16_64_OAarea_forth_selection_plus_forth_students_chosen = (pd.concat([df_persons_NE_inactive_16_64_OAarea_forth_selection, globals()[f"df_{gender}_forth_students_chosen"]]))
        
        #Remove duplicates
        df_persons_NE_inactive_16_64_OAarea_fifth_selection = df_persons_NE_inactive_16_64_OAarea_forth_selection_plus_forth_students_chosen.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
        # Select only those of a specific gender
        df_persons_NE_inactive_16_64_OAarea_fifth_selection_gender = df_persons_NE_inactive_16_64_OAarea_fifth_selection.loc[(df_persons_NE_inactive_16_64_OAarea_fifth_selection['Sex'] == gender)]
        
        
        if fifth_selection_inactive_students_2019 > 0:
                
            globals()[f"df_{gender}_fifth_students_chosen"] = df_persons_NE_inactive_16_64_OAarea_fifth_selection_gender.sort_values('Age', ascending = True).head(fifth_selection_inactive_students_2019)

        
        else:
                    
            globals()[f"df_{gender}_fifth_students_chosen"] = pd.DataFrame() 
            
        
        #Concatenate the selected dataframes with the selected students
        globals()[f"df_{gender}_inactive_students"] = (pd.concat([globals()[f"df_{gender}_first_students_chosen"], globals()[f"df_{gender}_second_students_chosen"], globals()[f"df_{gender}_third_students_chosen"], globals()[f"df_{gender}_forth_students_chosen"], globals()[f"df_{gender}_fifth_students_chosen"]]))
        
        
        #Append the dataframe into the temporal list
        persons_NE_inactive_students_chosen_all_list.append(globals()[f"df_{gender}_inactive_students"])
    
    
# Concatenate the selected dataframes with the selected students
df_inactive_students_all = pd.concat(persons_NE_inactive_students_chosen_all_list, axis=0, ignore_index=True)
            
        
print('The code has finished. Check results!')       
        
        

Number of OA areas in iteration:  (1, 8802)
Number of OA areas in iteration:  (2, 8802)
Number of OA areas in iteration:  (3, 8802)
Number of OA areas in iteration:  (4, 8802)
Number of OA areas in iteration:  (5, 8802)
Number of OA areas in iteration:  (6, 8802)
Number of OA areas in iteration:  (7, 8802)
Number of OA areas in iteration:  (8, 8802)
Number of OA areas in iteration:  (9, 8802)
Number of OA areas in iteration:  (10, 8802)
Number of OA areas in iteration:  (11, 8802)
Number of OA areas in iteration:  (12, 8802)
Number of OA areas in iteration:  (13, 8802)
Number of OA areas in iteration:  (14, 8802)
Number of OA areas in iteration:  (15, 8802)
Number of OA areas in iteration:  (16, 8802)
Number of OA areas in iteration:  (17, 8802)
Number of OA areas in iteration:  (18, 8802)
Number of OA areas in iteration:  (19, 8802)
Number of OA areas in iteration:  (20, 8802)
Number of OA areas in iteration:  (21, 8802)
Number of OA areas in iteration:  (22, 8802)
Number of OA areas 

Number of OA areas in iteration:  (183, 8802)
Number of OA areas in iteration:  (184, 8802)
Number of OA areas in iteration:  (185, 8802)
Number of OA areas in iteration:  (186, 8802)
Number of OA areas in iteration:  (187, 8802)
Number of OA areas in iteration:  (188, 8802)
Number of OA areas in iteration:  (189, 8802)
Number of OA areas in iteration:  (190, 8802)
Number of OA areas in iteration:  (191, 8802)
Number of OA areas in iteration:  (192, 8802)
Number of OA areas in iteration:  (193, 8802)
Number of OA areas in iteration:  (194, 8802)
Number of OA areas in iteration:  (195, 8802)
Number of OA areas in iteration:  (196, 8802)
Number of OA areas in iteration:  (197, 8802)
Number of OA areas in iteration:  (198, 8802)
Number of OA areas in iteration:  (199, 8802)
Number of OA areas in iteration:  (200, 8802)
Number of OA areas in iteration:  (201, 8802)
Number of OA areas in iteration:  (202, 8802)
Number of OA areas in iteration:  (203, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (362, 8802)
Number of OA areas in iteration:  (363, 8802)
Number of OA areas in iteration:  (364, 8802)
Number of OA areas in iteration:  (365, 8802)
Number of OA areas in iteration:  (366, 8802)
Number of OA areas in iteration:  (367, 8802)
Number of OA areas in iteration:  (368, 8802)
Number of OA areas in iteration:  (369, 8802)
Number of OA areas in iteration:  (370, 8802)
Number of OA areas in iteration:  (371, 8802)
Number of OA areas in iteration:  (372, 8802)
Number of OA areas in iteration:  (373, 8802)
Number of OA areas in iteration:  (374, 8802)
Number of OA areas in iteration:  (375, 8802)
Number of OA areas in iteration:  (376, 8802)
Number of OA areas in iteration:  (377, 8802)
Number of OA areas in iteration:  (378, 8802)
Number of OA areas in iteration:  (379, 8802)
Number of OA areas in iteration:  (380, 8802)
Number of OA areas in iteration:  (381, 8802)
Number of OA areas in iteration:  (382, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (541, 8802)
Number of OA areas in iteration:  (542, 8802)
Number of OA areas in iteration:  (543, 8802)
Number of OA areas in iteration:  (544, 8802)
Number of OA areas in iteration:  (545, 8802)
Number of OA areas in iteration:  (546, 8802)
Number of OA areas in iteration:  (547, 8802)
Number of OA areas in iteration:  (548, 8802)
Number of OA areas in iteration:  (549, 8802)
Number of OA areas in iteration:  (550, 8802)
Number of OA areas in iteration:  (551, 8802)
Number of OA areas in iteration:  (552, 8802)
Number of OA areas in iteration:  (553, 8802)
Number of OA areas in iteration:  (554, 8802)
Number of OA areas in iteration:  (555, 8802)
Number of OA areas in iteration:  (556, 8802)
Number of OA areas in iteration:  (557, 8802)
Number of OA areas in iteration:  (558, 8802)
Number of OA areas in iteration:  (559, 8802)
Number of OA areas in iteration:  (560, 8802)
Number of OA areas in iteration:  (561, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (722, 8802)
Number of OA areas in iteration:  (723, 8802)
Number of OA areas in iteration:  (724, 8802)
Number of OA areas in iteration:  (725, 8802)
Number of OA areas in iteration:  (726, 8802)
Number of OA areas in iteration:  (727, 8802)
Number of OA areas in iteration:  (728, 8802)
Number of OA areas in iteration:  (729, 8802)
Number of OA areas in iteration:  (730, 8802)
Number of OA areas in iteration:  (731, 8802)
Number of OA areas in iteration:  (732, 8802)
Number of OA areas in iteration:  (733, 8802)
Number of OA areas in iteration:  (734, 8802)
Number of OA areas in iteration:  (735, 8802)
Number of OA areas in iteration:  (736, 8802)
Number of OA areas in iteration:  (737, 8802)
Number of OA areas in iteration:  (738, 8802)
Number of OA areas in iteration:  (739, 8802)
Number of OA areas in iteration:  (740, 8802)
Number of OA areas in iteration:  (741, 8802)
Number of OA areas in iteration:  (742, 8802)
Number of OA areas in iteration:  

...

Number of OA areas in iteration:  (7949, 8802)
Number of OA areas in iteration:  (7950, 8802)
Number of OA areas in iteration:  (7951, 8802)
Number of OA areas in iteration:  (7952, 8802)
Number of OA areas in iteration:  (7953, 8802)
Number of OA areas in iteration:  (7954, 8802)
Number of OA areas in iteration:  (7955, 8802)
Number of OA areas in iteration:  (7956, 8802)
Number of OA areas in iteration:  (7957, 8802)
Number of OA areas in iteration:  (7958, 8802)
Number of OA areas in iteration:  (7959, 8802)
Number of OA areas in iteration:  (7960, 8802)
Number of OA areas in iteration:  (7961, 8802)
Number of OA areas in iteration:  (7962, 8802)
Number of OA areas in iteration:  (7963, 8802)
Number of OA areas in iteration:  (7964, 8802)
Number of OA areas in iteration:  (7965, 8802)
Number of OA areas in iteration:  (7966, 8802)
Number of OA areas in iteration:  (7967, 8802)
Number of OA areas in iteration:  (7968, 8802)
Number of OA areas in iteration:  (7969, 8802)
Number of OA 

Number of OA areas in iteration:  (8126, 8802)
Number of OA areas in iteration:  (8127, 8802)
Number of OA areas in iteration:  (8128, 8802)
Number of OA areas in iteration:  (8129, 8802)
Number of OA areas in iteration:  (8130, 8802)
Number of OA areas in iteration:  (8131, 8802)
Number of OA areas in iteration:  (8132, 8802)
Number of OA areas in iteration:  (8133, 8802)
Number of OA areas in iteration:  (8134, 8802)
Number of OA areas in iteration:  (8135, 8802)
Number of OA areas in iteration:  (8136, 8802)
Number of OA areas in iteration:  (8137, 8802)
Number of OA areas in iteration:  (8138, 8802)
Number of OA areas in iteration:  (8139, 8802)
Number of OA areas in iteration:  (8140, 8802)
Number of OA areas in iteration:  (8141, 8802)
Number of OA areas in iteration:  (8142, 8802)
Number of OA areas in iteration:  (8143, 8802)
Number of OA areas in iteration:  (8144, 8802)
Number of OA areas in iteration:  (8145, 8802)
Number of OA areas in iteration:  (8146, 8802)
Number of OA 

Number of OA areas in iteration:  (8304, 8802)
Number of OA areas in iteration:  (8305, 8802)
Number of OA areas in iteration:  (8306, 8802)
Number of OA areas in iteration:  (8307, 8802)
Number of OA areas in iteration:  (8308, 8802)
Number of OA areas in iteration:  (8309, 8802)
Number of OA areas in iteration:  (8310, 8802)
Number of OA areas in iteration:  (8311, 8802)
Number of OA areas in iteration:  (8312, 8802)
Number of OA areas in iteration:  (8313, 8802)
Number of OA areas in iteration:  (8314, 8802)
Number of OA areas in iteration:  (8315, 8802)
Number of OA areas in iteration:  (8316, 8802)
Number of OA areas in iteration:  (8317, 8802)
Number of OA areas in iteration:  (8318, 8802)
Number of OA areas in iteration:  (8319, 8802)
Number of OA areas in iteration:  (8320, 8802)
Number of OA areas in iteration:  (8321, 8802)
Number of OA areas in iteration:  (8322, 8802)
Number of OA areas in iteration:  (8323, 8802)
Number of OA areas in iteration:  (8324, 8802)
Number of OA 

Number of OA areas in iteration:  (8479, 8802)
Number of OA areas in iteration:  (8480, 8802)
Number of OA areas in iteration:  (8481, 8802)
Number of OA areas in iteration:  (8482, 8802)
Number of OA areas in iteration:  (8483, 8802)
Number of OA areas in iteration:  (8484, 8802)
Number of OA areas in iteration:  (8485, 8802)
Number of OA areas in iteration:  (8486, 8802)
Number of OA areas in iteration:  (8487, 8802)
Number of OA areas in iteration:  (8488, 8802)
Number of OA areas in iteration:  (8489, 8802)
Number of OA areas in iteration:  (8490, 8802)
Number of OA areas in iteration:  (8491, 8802)
Number of OA areas in iteration:  (8492, 8802)
Number of OA areas in iteration:  (8493, 8802)
Number of OA areas in iteration:  (8494, 8802)
Number of OA areas in iteration:  (8495, 8802)
Number of OA areas in iteration:  (8496, 8802)
Number of OA areas in iteration:  (8497, 8802)
Number of OA areas in iteration:  (8498, 8802)
Number of OA areas in iteration:  (8499, 8802)
Number of OA 

Number of OA areas in iteration:  (8655, 8802)
Number of OA areas in iteration:  (8656, 8802)
Number of OA areas in iteration:  (8657, 8802)
Number of OA areas in iteration:  (8658, 8802)
Number of OA areas in iteration:  (8659, 8802)
Number of OA areas in iteration:  (8660, 8802)
Number of OA areas in iteration:  (8661, 8802)
Number of OA areas in iteration:  (8662, 8802)
Number of OA areas in iteration:  (8663, 8802)
Number of OA areas in iteration:  (8664, 8802)
Number of OA areas in iteration:  (8665, 8802)
Number of OA areas in iteration:  (8666, 8802)
Number of OA areas in iteration:  (8667, 8802)
Number of OA areas in iteration:  (8668, 8802)
Number of OA areas in iteration:  (8669, 8802)
Number of OA areas in iteration:  (8670, 8802)
Number of OA areas in iteration:  (8671, 8802)
Number of OA areas in iteration:  (8672, 8802)
Number of OA areas in iteration:  (8673, 8802)
Number of OA areas in iteration:  (8674, 8802)
Number of OA areas in iteration:  (8675, 8802)
Number of OA 

In [14]:
# check the total number of students selected:
len(df_inactive_students_all)

102542

In [15]:
# Check if there are duplicates
df_inactive_students_all_1 = df_inactive_students_all

# Remove duplicates
df_inactive_students_all_1 = df_inactive_students_all.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_inactive_students_all_1)


102542

In [16]:
#####################################################
##   CHECKS:
    
# Calculate the % of students within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_students = len(df_inactive_students_all)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_students was', percentage_students)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_students_male = df_inactive_students_all.loc[(df_inactive_students_all['Sex'] == 1)]
percentage_males_students = len(df_persons_NE_inactive_students_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_students was', percentage_males_students)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_students_female = df_inactive_students_all.loc[(df_inactive_students_all['Sex'] == 2)]
percentage_females_students = len(df_persons_NE_inactive_students_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_students was', percentage_females_students)
    

# Calculate the % proportion of males that are students
percentage_proportion_male = len(df_persons_NE_inactive_students_male)/(len(df_inactive_students_all)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are students
percentage_proportion_female = len(df_persons_NE_inactive_students_female)/(len(df_inactive_students_all)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


Value percentage_students was 25.14695748348951
Value percentage_males_students was 34.16475376860639
Value percentage_females_students was 18.76243491955785
Value percentage_proportion_male was 56.31643619199937
Value percentage_proportion_female was 43.68356380800063
Code finished. Check the results!


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_student_conversor" parameter value.

In [18]:
# Save dataframe
df_inactive_students_all_export20220225_correct = df_inactive_students_all
df_inactive_students_all_export20220225_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_inactive_students_all_export20220225_correct.csv', encoding='utf-8', header=True)


## LOOKING AFTER FAMILY / HOME

In [22]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_plus_students = (pd.concat([df_persons_NE_inactive_16_64, df_inactive_students_all]))

# Remove duplicates and keep only those who were not assigned a driving licence
df_persons_NE_inactive_16_64_NO_students = df_persons_NE_inactive_16_64_plus_students.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_NO_students)

305229

In [23]:
# check the remaining agents
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all)

305229

In [26]:
## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
## Run first females because there are more to be assinged in this category.
gender_list = [2,1]

#Create an empty list where the small blocks of dataframes of "LOOKING AFTER" people will be stored
people_NE_inactive_lookingafter_all_list = []


# % inactive conversor value 2011 to 2019
## This value is converting the % of inactive people (each type) from 2011 to 2019 based on data from "Regional labour market statistics: HI01 Headline indicators for the North East"
inactive_loookingafter_conversor = 1.75

#  Relationship between male and female people per reason of inactivity (male + female =1)
## This values are coming from Regional labour market statistics:HI01 Headline indicators for the North East. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_lookingafter = 0.202

# Females
percentage_2_lookingafter = 0.798


#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:
    
    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
            
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]
    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2019 = len(df_persons_NE_inactive_16_64_OAarea)

    
    # Calculate the ratio of people 2019 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2019_2011 = total_inactive_in_OA_2019/total_QS601EW_inactive_in_OA_2011
    else:
        ratio_people_2019_2011 = 1
        
        
    # Get the VALUE of the number of people living in the seleted OA area that are inactive STUDENTS from table QS601EW
    total_QS601EW_inactive_lookingafter_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Looking_after_home_or_family')]
    
    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to student ("total_QS601EW_inactive_students_in_OA_2011") 
    # 2. the "inactive_student_conversor" value
    # 3. the ratio of people 2019 vs 2011 ("ratio_people_2019_2011")
    
    # New value for 2019 =  (Value from 2011 (table QS601EW)) * ("inactive_student_conversor"  value) * (ratio of people 2019 vs 2011 ("ratio_people_2019_2011")) 
    ## This value is the number of people that will be assigned "STUDENT" in the OA area selected
    inactive_lookingafter_2019 = int(round(total_QS601EW_inactive_lookingafter_in_OA_2011 * inactive_loookingafter_conversor * ratio_people_2019_2011,0)) 
    

    
    #################################################################################################################
    ## START IDENTIFYING THE "LOOKING AFTER HOME /  FAMILY" ATTRIBUTE VALUE
    
    # SELECT people as 'LOOKING AFTER HOME/FAMILY' BASED ON THEIR SOCIOECONOMIC CHARACTERISTICS:
    ## First selection:
    ### 1. Marital status: Married or Couple
    ### 2. Children dependency: True
    ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    
    # In case there are still leftovers:
    ## Second selection:
    ### 1. Marital status: Married or Couple
    ### 2. Children dependency: False
    ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    ### 4. Age: sort age by 9asecnding order = False) and choose the top 'n values required
    
    # In case there are still leftovers:
    ## Third selection:
    ### 1. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    ### 2. Marital status: any type (married, couple or single)
    ### Age: no age requirements.
    

    #Empty list where households ID's will be included once one gender is ran
    # This is to avoid that more than one person is assigned as "looking after home/family" per household
    all_HID_AreaOA_x_list = []
    
    
    for gender in gender_list:
        
        # In order to avoid having more than one person per household in the category "LOOKING AFTER FAMILY / HOME",
        # After the first iteration (female loop) and when the second loop starts (male), 
        # If a male belongs to a household were a female was selected, then the male is not considered to be in the category "LOOKING AFTER FAMILY / HOME"
        # Create a new dataframe removing the rows which household IDs have already assigned a person as "LOOKING AFTER HOME/FAMILY"
        # The values to be removed are coming from the list "all_HID_AreaOA_x_list"
        # For females, the list is always empty
        # For males, this list contains the HID_AreaOA_x of females that were already assigned as "LOOKING AFTER HOME/FAMILY"
        # So, those males that live in those households will NOT be considered to be assigned as "LOOKING AFTER HOME/FAMILY"
        # This is to avoid that a household has more than one person in this category.
        df_persons_NE_inactive_16_64_NO_students_unique = df_persons_NE_inactive_16_64_NO_students[~df_persons_NE_inactive_16_64_NO_students.HID_AreaOA_x.isin(all_HID_AreaOA_x_list)]
        
        
        ## FIRST SELECTION:
        ### 1. Marital status: Married or Couple
        ### 2. Children dependency: True
        ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
    
        # Store in an array the possible marital_status to be considered first
        marital_status_array = ['Married', 'Couple']
        
        
        
        # FRIST SELECTION: Select those people aged 16-64, married/couple with children dependency    
        df_persons_NE_inactive_16_64_married_or_couples_childrendepend = df_persons_NE_inactive_16_64_NO_students_unique.loc[(df_persons_NE_inactive_16_64_NO_students_unique['Marital_status'].isin(marital_status_array)) & (df_persons_NE_inactive_16_64_NO_students_unique['Area_OA_x'] == OA_area) & (df_persons_NE_inactive_16_64_NO_students_unique['Children_dependency'] == True) & (df_persons_NE_inactive_16_64_NO_students_unique['Sex'] == gender)]
        
        if (len(df_persons_NE_inactive_16_64_married_or_couples_childrendepend) > 0):
            
            if ((int(round(inactive_lookingafter_2019 * globals()[f"percentage_{gender}_lookingafter"],0))) <= len(df_persons_NE_inactive_16_64_married_or_couples_childrendepend)):
                
                # Select randomly the number of people that will be assigned as 'LOOKING AFTER FAMILY / HOME'               
                df_first_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_childrendepend.sample(int(round(inactive_lookingafter_2019 * globals()[f"percentage_{gender}_lookingafter"],0)))
                
        
            else:
                
                # Select all people that will be assigned as "LOOKING AFTER HOME / FAMILY"
                df_first_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_childrendepend
                
                # Not all 'LOOKING AFTER FAMILY / HOME' people have already been selected, just a few of them.
                # Second selection should be run                                                                                                       
                                                                                                                     
        # If there is not any person with the selected charateristics in the dataframe:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            df_first_selection_lookingafter =  pd.DataFrame()
                
            # None persons have already been selected.
            # Second selection should be run.

                                                                                                                        
        ## SECOND SELECTION:                                                                                                               
        ### 1. Marital status: Married or Couple
        ### 2. Children dependency: False
        ### 3. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
        ### 4. Age: sort age by 9asecnding order = False) and choose the top 'n values required
        
        # Remaining NUMBER of people to be assigned as "LOOKING AFTER FAMILY / HOME":
        second_selection_lookingafter_students_2019 = (int(round(inactive_lookingafter_2019 * globals()[f"percentage_{gender}_lookingafter"],0)) - len(df_first_selection_lookingafter))
        
        # Concatenate 
        df_persons_NE_inactive_16_64_NO_students_plus_first_selection_lookingafter = (pd.concat([df_persons_NE_inactive_16_64_NO_students, df_first_selection_lookingafter]))
                
        # Remove duplicates
        df_persons_NE_inactive_16_64_NO_students_second_selection = df_persons_NE_inactive_16_64_NO_students_plus_first_selection_lookingafter.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
                                                                                                                        
                                                                                                                        
        # Select those that are aged 16-64 and married or couple without any children dependency                                                                                                                     
        df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend = df_persons_NE_inactive_16_64_NO_students_second_selection.loc[(df_persons_NE_inactive_16_64_NO_students_second_selection['Marital_status'].isin(marital_status_array)) & (df_persons_NE_inactive_16_64_NO_students_second_selection['Area_OA_x'] == OA_area) & (df_persons_NE_inactive_16_64_NO_students_second_selection['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_second_selection['Children_dependency'] == False)]
                                                                                                                         
                                                                                                                      
        if (second_selection_lookingafter_students_2019 > 0):
        
            if (len(df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend) > 0):
                                                                                                                        
                if ((second_selection_lookingafter_students_2019) <= len(df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend)):
                
                    # Select the number of people that will be assigned as 'LOOKING AFTER FAMILY / HOME' sorted by age              
                    df_second_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend.sort_values('Age', ascending = False).head(second_selection_lookingafter_students_2019)
                                                                                            
                else:
                
                    # Select all people that will be assigned as "LOOKING AFTER HOME / FAMILY"
                    df_second_selection_lookingafter = df_persons_NE_inactive_16_64_married_or_couples_no_childrendepend
                                                                                                                        
            # If there is not any person with the selected charateristics in the dataframe:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                df_second_selection_lookingafter =  pd.DataFrame()                                                                                                            
                                                                                                                        
                                                                                                                        
        # If there remaining number of people to be selected (second_selection_lookingafter_students_2019) is zero:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            df_second_selection_lookingafter =  pd.DataFrame() 
                                                                                                                        
                                                                                                                        
                                                                                                                        
        ## THIRD SELECTION                                                                                                                
        ### 1. Sex: proportions based on variables "percentage_male_lookingafter" and "percentage_female_lookingafter" assigned previously
        ### 2. Marital status: any type (married, couple or single)
        ### Age: no age requirements.
                                                                                                                        
        # Remaining NUMBER of people to be assigned as "LOOKING AFTER FAMILY / HOME":
        third_selection_lookingafter_students_2019 = (int(round(inactive_lookingafter_2019 * globals()[f"percentage_{gender}_lookingafter"],0)) - len(df_first_selection_lookingafter) - len(df_second_selection_lookingafter))
        
        # Concatenate 
        df_persons_NE_inactive_16_64_NO_students_second_selection_plus_second_selection_lookingafter = (pd.concat([df_persons_NE_inactive_16_64_NO_students_second_selection, df_second_selection_lookingafter]))
                
        # Remove duplicates
        df_persons_NE_inactive_16_64_NO_students_third_selection = df_persons_NE_inactive_16_64_NO_students_second_selection_plus_second_selection_lookingafter.drop_duplicates(subset='PID_AreaMSOA', keep = False)
        
                                                                                                                   
        # Select those that are aged 16-64 (no marital status and age requirements)                                                                                                                        
        df_persons_NE_inactive_16_64_remaining = df_persons_NE_inactive_16_64_NO_students_third_selection.loc[(df_persons_NE_inactive_16_64_NO_students_third_selection['Area_OA_x'] == OA_area) & (df_persons_NE_inactive_16_64_NO_students_third_selection['Sex'] == gender)]
                                                                                                                          
                                                                                                                        
        if (third_selection_lookingafter_students_2019 > 0): 
           
            if (len(df_persons_NE_inactive_16_64_remaining) > 0): 
                                                                                                                        
                if ((third_selection_lookingafter_students_2019) <= len(df_persons_NE_inactive_16_64_remaining)):
                
                    # Select the number of people that will be assigned as 'LOOKING AFTER FAMILY / HOME' based on the value stored in "third_selection_lookingafter_students_2019"          
                    df_third_selection_lookingafter = df_persons_NE_inactive_16_64_remaining.sample(third_selection_lookingafter_students_2019)
                                                                                                                                 
                else:
                
                    # Select all people that will be assigned as "LOOKING AFTER HOME / FAMILY"
                    df_third_selection_lookingafter = df_persons_NE_inactive_16_64_remaining   
                                                                                                                        
            # If there is not any person with the selected charateristics in the dataframe:
            else:
                
                #If the dataframe does not have any rows, then it will be empty.
                df_third_selection_lookingafter =  pd.DataFrame() 
                                                                                                                        
        # If there remaining number of people to be selected (third_selection_lookingafter_students_2019) is zero:
        else:
                
            #If the dataframe does not have any rows, then it will be empty.
            df_third_selection_lookingafter =  pd.DataFrame()                                                                                                                 
                                                                                                                        
                                                                                                                        
                                                                                                                        
        #Concatenate the selected dataframes with the selected students
        globals()[f"df_{gender}_inactive_looking_after"] = (pd.concat([df_first_selection_lookingafter, df_second_selection_lookingafter, df_third_selection_lookingafter]))
        
        
        # Include in the list the HID_AreaOA_x values in order to avoid that another person of the other gender can be categorised as "Looking after family/home"
        #Append the dataframe into the temporal list
        if len(globals()[f"df_{gender}_inactive_looking_after"]) > 0:
            all_HID_AreaOA_x_list = globals()[f"df_{gender}_inactive_looking_after"]['HID_AreaOA_x'].tolist()
        

        people_NE_inactive_lookingafter_all_list.append(globals()[f"df_{gender}_inactive_looking_after"])
                                                                                                                 

# Concatenate the selected dataframes with the selected looking after family/home people
df_inactive_lookingafter_all_22feb_3 = pd.concat(people_NE_inactive_lookingafter_all_list, axis=0, ignore_index=True)
                                                                                                                        
                                                                                                                     
print('Code has finished. Check the results!')  

Number of OA areas in iteration:  (1, 8802)
Number of OA areas in iteration:  (2, 8802)
Number of OA areas in iteration:  (3, 8802)
Number of OA areas in iteration:  (4, 8802)
Number of OA areas in iteration:  (5, 8802)
Number of OA areas in iteration:  (6, 8802)
Number of OA areas in iteration:  (7, 8802)
Number of OA areas in iteration:  (8, 8802)
Number of OA areas in iteration:  (9, 8802)
Number of OA areas in iteration:  (10, 8802)
Number of OA areas in iteration:  (11, 8802)
Number of OA areas in iteration:  (12, 8802)
Number of OA areas in iteration:  (13, 8802)
Number of OA areas in iteration:  (14, 8802)
Number of OA areas in iteration:  (15, 8802)
Number of OA areas in iteration:  (16, 8802)
Number of OA areas in iteration:  (17, 8802)
Number of OA areas in iteration:  (18, 8802)
Number of OA areas in iteration:  (19, 8802)
Number of OA areas in iteration:  (20, 8802)
Number of OA areas in iteration:  (21, 8802)
Number of OA areas in iteration:  (22, 8802)
Number of OA areas 

Number of OA areas in iteration:  (182, 8802)
Number of OA areas in iteration:  (183, 8802)
Number of OA areas in iteration:  (184, 8802)
Number of OA areas in iteration:  (185, 8802)
Number of OA areas in iteration:  (186, 8802)
Number of OA areas in iteration:  (187, 8802)
Number of OA areas in iteration:  (188, 8802)
Number of OA areas in iteration:  (189, 8802)
Number of OA areas in iteration:  (190, 8802)
Number of OA areas in iteration:  (191, 8802)
Number of OA areas in iteration:  (192, 8802)
Number of OA areas in iteration:  (193, 8802)
Number of OA areas in iteration:  (194, 8802)
Number of OA areas in iteration:  (195, 8802)
Number of OA areas in iteration:  (196, 8802)
Number of OA areas in iteration:  (197, 8802)
Number of OA areas in iteration:  (198, 8802)
Number of OA areas in iteration:  (199, 8802)
Number of OA areas in iteration:  (200, 8802)
Number of OA areas in iteration:  (201, 8802)
Number of OA areas in iteration:  (202, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (361, 8802)
Number of OA areas in iteration:  (362, 8802)
Number of OA areas in iteration:  (363, 8802)
Number of OA areas in iteration:  (364, 8802)
Number of OA areas in iteration:  (365, 8802)
Number of OA areas in iteration:  (366, 8802)
Number of OA areas in iteration:  (367, 8802)
Number of OA areas in iteration:  (368, 8802)
Number of OA areas in iteration:  (369, 8802)
Number of OA areas in iteration:  (370, 8802)
Number of OA areas in iteration:  (371, 8802)
Number of OA areas in iteration:  (372, 8802)
Number of OA areas in iteration:  (373, 8802)
Number of OA areas in iteration:  (374, 8802)
Number of OA areas in iteration:  (375, 8802)
Number of OA areas in iteration:  (376, 8802)
Number of OA areas in iteration:  (377, 8802)
Number of OA areas in iteration:  (378, 8802)
Number of OA areas in iteration:  (379, 8802)
Number of OA areas in iteration:  (380, 8802)
Number of OA areas in iteration:  (381, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (540, 8802)
Number of OA areas in iteration:  (541, 8802)
Number of OA areas in iteration:  (542, 8802)
Number of OA areas in iteration:  (543, 8802)
Number of OA areas in iteration:  (544, 8802)
Number of OA areas in iteration:  (545, 8802)
Number of OA areas in iteration:  (546, 8802)
Number of OA areas in iteration:  (547, 8802)
Number of OA areas in iteration:  (548, 8802)
Number of OA areas in iteration:  (549, 8802)
Number of OA areas in iteration:  (550, 8802)
Number of OA areas in iteration:  (551, 8802)
Number of OA areas in iteration:  (552, 8802)
Number of OA areas in iteration:  (553, 8802)
Number of OA areas in iteration:  (554, 8802)
Number of OA areas in iteration:  (555, 8802)
Number of OA areas in iteration:  (556, 8802)
Number of OA areas in iteration:  (557, 8802)
Number of OA areas in iteration:  (558, 8802)
Number of OA areas in iteration:  (559, 8802)
Number of OA areas in iteration:  (560, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (719, 8802)
Number of OA areas in iteration:  (720, 8802)
Number of OA areas in iteration:  (721, 8802)
Number of OA areas in iteration:  (722, 8802)
Number of OA areas in iteration:  (723, 8802)
Number of OA areas in iteration:  (724, 8802)
Number of OA areas in iteration:  (725, 8802)
Number of OA areas in iteration:  (726, 8802)
Number of OA areas in iteration:  (727, 8802)
Number of OA areas in iteration:  (728, 8802)
Number of OA areas in iteration:  (729, 8802)
Number of OA areas in iteration:  (730, 8802)
Number of OA areas in iteration:  (731, 8802)
Number of OA areas in iteration:  (732, 8802)
Number of OA areas in iteration:  (733, 8802)
Number of OA areas in iteration:  (734, 8802)
Number of OA areas in iteration:  (735, 8802)
Number of OA areas in iteration:  (736, 8802)
Number of OA areas in iteration:  (737, 8802)
Number of OA areas in iteration:  (738, 8802)
Number of OA areas in iteration:  (739, 8802)
Number of OA areas in iteration:  

...

Number of OA areas in iteration:  (8075, 8802)
Number of OA areas in iteration:  (8076, 8802)
Number of OA areas in iteration:  (8077, 8802)
Number of OA areas in iteration:  (8078, 8802)
Number of OA areas in iteration:  (8079, 8802)
Number of OA areas in iteration:  (8080, 8802)
Number of OA areas in iteration:  (8081, 8802)
Number of OA areas in iteration:  (8082, 8802)
Number of OA areas in iteration:  (8083, 8802)
Number of OA areas in iteration:  (8084, 8802)
Number of OA areas in iteration:  (8085, 8802)
Number of OA areas in iteration:  (8086, 8802)
Number of OA areas in iteration:  (8087, 8802)
Number of OA areas in iteration:  (8088, 8802)
Number of OA areas in iteration:  (8089, 8802)
Number of OA areas in iteration:  (8090, 8802)
Number of OA areas in iteration:  (8091, 8802)
Number of OA areas in iteration:  (8092, 8802)
Number of OA areas in iteration:  (8093, 8802)
Number of OA areas in iteration:  (8094, 8802)
Number of OA areas in iteration:  (8095, 8802)
Number of OA 

Number of OA areas in iteration:  (8250, 8802)
Number of OA areas in iteration:  (8251, 8802)
Number of OA areas in iteration:  (8252, 8802)
Number of OA areas in iteration:  (8253, 8802)
Number of OA areas in iteration:  (8254, 8802)
Number of OA areas in iteration:  (8255, 8802)
Number of OA areas in iteration:  (8256, 8802)
Number of OA areas in iteration:  (8257, 8802)
Number of OA areas in iteration:  (8258, 8802)
Number of OA areas in iteration:  (8259, 8802)
Number of OA areas in iteration:  (8260, 8802)
Number of OA areas in iteration:  (8261, 8802)
Number of OA areas in iteration:  (8262, 8802)
Number of OA areas in iteration:  (8263, 8802)
Number of OA areas in iteration:  (8264, 8802)
Number of OA areas in iteration:  (8265, 8802)
Number of OA areas in iteration:  (8266, 8802)
Number of OA areas in iteration:  (8267, 8802)
Number of OA areas in iteration:  (8268, 8802)
Number of OA areas in iteration:  (8269, 8802)
Number of OA areas in iteration:  (8270, 8802)
Number of OA 

Number of OA areas in iteration:  (8425, 8802)
Number of OA areas in iteration:  (8426, 8802)
Number of OA areas in iteration:  (8427, 8802)
Number of OA areas in iteration:  (8428, 8802)
Number of OA areas in iteration:  (8429, 8802)
Number of OA areas in iteration:  (8430, 8802)
Number of OA areas in iteration:  (8431, 8802)
Number of OA areas in iteration:  (8432, 8802)
Number of OA areas in iteration:  (8433, 8802)
Number of OA areas in iteration:  (8434, 8802)
Number of OA areas in iteration:  (8435, 8802)
Number of OA areas in iteration:  (8436, 8802)
Number of OA areas in iteration:  (8437, 8802)
Number of OA areas in iteration:  (8438, 8802)
Number of OA areas in iteration:  (8439, 8802)
Number of OA areas in iteration:  (8440, 8802)
Number of OA areas in iteration:  (8441, 8802)
Number of OA areas in iteration:  (8442, 8802)
Number of OA areas in iteration:  (8443, 8802)
Number of OA areas in iteration:  (8444, 8802)
Number of OA areas in iteration:  (8445, 8802)
Number of OA 

Number of OA areas in iteration:  (8600, 8802)
Number of OA areas in iteration:  (8601, 8802)
Number of OA areas in iteration:  (8602, 8802)
Number of OA areas in iteration:  (8603, 8802)
Number of OA areas in iteration:  (8604, 8802)
Number of OA areas in iteration:  (8605, 8802)
Number of OA areas in iteration:  (8606, 8802)
Number of OA areas in iteration:  (8607, 8802)
Number of OA areas in iteration:  (8608, 8802)
Number of OA areas in iteration:  (8609, 8802)
Number of OA areas in iteration:  (8610, 8802)
Number of OA areas in iteration:  (8611, 8802)
Number of OA areas in iteration:  (8612, 8802)
Number of OA areas in iteration:  (8613, 8802)
Number of OA areas in iteration:  (8614, 8802)
Number of OA areas in iteration:  (8615, 8802)
Number of OA areas in iteration:  (8616, 8802)
Number of OA areas in iteration:  (8617, 8802)
Number of OA areas in iteration:  (8618, 8802)
Number of OA areas in iteration:  (8619, 8802)
Number of OA areas in iteration:  (8620, 8802)
Number of OA 

Number of OA areas in iteration:  (8775, 8802)
Number of OA areas in iteration:  (8776, 8802)
Number of OA areas in iteration:  (8777, 8802)
Number of OA areas in iteration:  (8778, 8802)
Number of OA areas in iteration:  (8779, 8802)
Number of OA areas in iteration:  (8780, 8802)
Number of OA areas in iteration:  (8781, 8802)
Number of OA areas in iteration:  (8782, 8802)
Number of OA areas in iteration:  (8783, 8802)
Number of OA areas in iteration:  (8784, 8802)
Number of OA areas in iteration:  (8785, 8802)
Number of OA areas in iteration:  (8786, 8802)
Number of OA areas in iteration:  (8787, 8802)
Number of OA areas in iteration:  (8788, 8802)
Number of OA areas in iteration:  (8789, 8802)
Number of OA areas in iteration:  (8790, 8802)
Number of OA areas in iteration:  (8791, 8802)
Number of OA areas in iteration:  (8792, 8802)
Number of OA areas in iteration:  (8793, 8802)
Number of OA areas in iteration:  (8794, 8802)
Number of OA areas in iteration:  (8795, 8802)
Number of OA 

In [27]:
# check the number of peoples selected as inactive looking after family/home
len(df_inactive_lookingafter_all_22feb_3)

87075

In [28]:
# check if there are duplicates
df_inactive_lookingafter_all_22feb_3_1 = df_inactive_lookingafter_all_22feb_3

# Remove duplicates
df_inactive_lookingafter_all_22feb_3_1 = df_inactive_lookingafter_all_22feb_3_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_inactive_lookingafter_all_22feb_3_1)

87075

In [29]:
#####################################################
# df_inactive_lookingafter_all_22feb_3
##   CHECKS:
    
# Calculate the % of students within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_lookingafter = len(df_inactive_lookingafter_all_22feb_3)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_lookingafter was', percentage_lookingafter)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_lookingafter_male = df_inactive_lookingafter_all_22feb_3.loc[(df_inactive_lookingafter_all_22feb_3['Sex'] == 1)]
percentage_males_lookingafter = len(df_persons_NE_inactive_lookingafter_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_students was', percentage_males_lookingafter)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_lookingafter_female = df_inactive_lookingafter_all_22feb_3.loc[(df_inactive_lookingafter_all_22feb_3['Sex'] == 2)]
percentage_females_lookingafter = len(df_persons_NE_inactive_lookingafter_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_students was', percentage_females_lookingafter)
    

# Calculate the % proportion of males that are students
percentage_proportion_male = len(df_persons_NE_inactive_lookingafter_male)/(len(df_inactive_lookingafter_all_22feb_3)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are students
percentage_proportion_female = len(df_persons_NE_inactive_lookingafter_female)/(len(df_inactive_lookingafter_all_22feb_3)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


Value percentage_lookingafter was 21.353897162868375
Value percentage_males_students was 10.245048157701682
Value percentage_females_students was 29.218867150031624
Value percentage_proportion_male was 19.887453344817686
Value percentage_proportion_female was 80.11254665518231
Code finished. Check the results!


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_loookingafter_conversor" parameter value.

In [30]:
# Save dataframe
df_inactive_lookingafter_all_22feb_3_export20220225_correct = df_inactive_lookingafter_all_22feb_3
df_inactive_lookingafter_all_22feb_3_export20220225_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_inactive_lookingafter_all_22feb_3_export20220225_correct.csv', encoding='utf-8', header=True)


## Retired

In [32]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_NO_students_plus_lookingafter = (pd.concat([df_persons_NE_inactive_16_64_NO_students, df_inactive_lookingafter_all_22feb_3]))

# Remove duplicates and keep only those who were not assigned a driving licence
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter = df_persons_NE_inactive_16_64_NO_students_plus_lookingafter.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter)

218154

In [33]:
# Check the number of remaining people to be assigned as "Retired", "Sick" or "Other"
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3)

218154

In [34]:
# Calculate the differences between 
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3) - len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter)

0

In [35]:
# Start developing the code to identify INACTIVE RETIRED

## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
gender_list = [1,2]

#Create an empty list where the small blocks of dataframes of "LOOKING AFTER" people will be stored
people_NE_inactive_retired_all_list = []


# % inactive conversor value 2011 to 2019
## This value is converting the % of inactive people (each type) from 2011 to 2019 based on data from "Regional labour market statistics: HI01 Headline indicators for the North East"
inactive_retired_conversor = 0.34


#  Relationship between male and female people per reason of inactivity (male + female =1)
## This values are coming from Regional labour market statistics:HI01 Headline indicators for the North East. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_retired = 0.545

# Females
percentage_2_retired = 0.455




#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:

    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
    
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]
    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2019 = len(df_persons_NE_inactive_16_64_OAarea)
    
    
    # Calculate the ratio of people 2019 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2019_2011 = total_inactive_in_OA_2019/total_QS601EW_inactive_in_OA_2011
        #print('Ratio people 2019_2011')
        #print(ratio_people_2019_2011)
    else:
        ratio_people_2019_2011 = 1
    
    
    # Get the VALUE of the number of people living in the seleted OA area that are inactive RETIRED from table QS601EW
    total_QS601EW_inactive_retired_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Retired')]

    
    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to each category (student, looking after, retired...)
    # 2. the inactive_XXXX_conversor
    # 3. the ratio of people 2019 vs 2011 ("ratio_people_2019_2011") [CONSTANT VALUE FOR ALL CATEGORIES]
    

    # New value for 2019 =  (Value from 2011 (table QS601EW)) * ("inactive_retired_conversor"  value) * (ratio of people 2019 vs 2011 ("ratio_people_2019_2011")) 
    ## This value is the number of people that will be assigned "RETIRED" in the OA area selected
    inactive_retired_2019 = int(round(total_QS601EW_inactive_retired_in_OA_2011 * inactive_retired_conversor * ratio_people_2019_2011,0)) 

    
    #################################################################################################################
    ## START IDENTIFYING THE "LOOKING AFTER HOME /  FAMILY" ATTRIBUTE VALUE
    
    
    # SELECT people as 'LOOKING AFTER HOME/FAMILY' BASED ON THEIR SOCIOECONOMIC CHARACTERISTICS:
    ## First selection:
    ### 1. People aged 60-64
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously
    # In case there are still leftovers:
    ## Second selection:
    ### 1. People aged 55-59
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously
    ## Third selection:
    ### 1. People aged 50-54
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously
    ## Forth selection:
    ### 1. People aged < 50
    ### 2. Sex: proportions based on variables "percentage_male_retired" and "percentage_female_retired" assigned previously

    for gender in gender_list:
        
        # Dataframe for First selection
        df_people_inactive_60_64_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] >= 60) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        # Dataframe for Second selection
        df_people_inactive_55_59_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] >= 55) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] <= 59) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        # Dataframe for Third selection
        df_people_inactive_50_54_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] >= 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] <= 54) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        # Dataframe for Forth selection
        df_people_inactive_50less_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Age'] < 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter['Area_OA_x'] == OA_area)]
        
        
        # Start choosing people in each of the 4 selection options defined before AND considering the gender %
        ## males: 54.5%   = percentage_1_retired
        ## females: 45.5% = percentage_2_retired
        

        if (len(df_people_inactive_60_64_gender) > int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0))):
        
            ## First selection:
            df_first_selection_retired = df_people_inactive_60_64_gender.sample(int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0)))

            df_second_selection_retired = pd.DataFrame()
            df_third_selection_retired = pd.DataFrame()
            df_forth_selection_retired = pd.DataFrame()
            
        elif (len(df_people_inactive_60_64_gender) == int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0))):
            
            ## First selection:
            df_first_selection_retired = df_people_inactive_60_64_gender

            df_second_selection_retired = pd.DataFrame()
            df_third_selection_retired = pd.DataFrame()
            df_forth_selection_retired = pd.DataFrame()
            
            
        else:
            ## First selection:
            df_first_selection_retired = df_people_inactive_60_64_gender

            # There are still remaining some people to be assigned as "RETIRED"
            if (len(df_people_inactive_55_59_gender) > (int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired)):
        
                ## Second selection:
                df_second_selection_retired = df_people_inactive_55_59_gender.sample(int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0)) - len(df_first_selection_retired))

                df_third_selection_retired = pd.DataFrame()
                df_forth_selection_retired = pd.DataFrame()
            
            elif (len(df_people_inactive_55_59_gender) == (int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired)):
        
                ## Second selection:
                df_second_selection_retired = df_people_inactive_55_59_gender

                df_third_selection_retired = pd.DataFrame()
                df_forth_selection_retired = pd.DataFrame()
            
            
            else:
                ## Second selection:
                df_second_selection_retired = df_people_inactive_55_59_gender

                # There are still remaining some people to be assigned as "RETIRED"
                if (len(df_people_inactive_50_54_gender) > (int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired) - len(df_second_selection_retired)):
        
                    ## Third selection:
                    df_third_selection_retired = df_people_inactive_50_54_gender.sample(int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0)) - len(df_first_selection_retired) - len(df_second_selection_retired))

                    df_forth_selection_retired = pd.DataFrame()
            
                elif (len(df_people_inactive_50_54_gender) == (int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0))) - len(df_first_selection_retired) - len(df_second_selection_retired)):
        
                    ## Third selection:
                    df_third_selection_retired = df_people_inactive_50_54_gender

                    df_forth_selection_retired = pd.DataFrame()
            
                else:
                    ## Third selection:
                    df_third_selection_retired = df_people_inactive_50_54_gender

                    
                    # Forth and final selection:                     
                    df_forth_selection_retired = df_people_inactive_50less_gender.sort_values('Age', ascending = False).head(int(round(inactive_retired_2019 * globals()[f"percentage_{gender}_retired"],0)) - len(df_first_selection_retired) - len(df_second_selection_retired) - len(df_third_selection_retired))

        
        # Once all possible ways to select people have been computed, there is a need to concatenate all dataframes:
        df_people_NE_inactive_retired = (pd.concat([df_first_selection_retired, df_second_selection_retired, df_third_selection_retired, df_forth_selection_retired]))
 
        #Append the dataframe into the temporal list
        people_NE_inactive_retired_all_list.append(df_people_NE_inactive_retired)
        

#concatenate all people "looking after home/family" in one dataframe
df_people_NE_inactive_retired_all_v2 = pd.concat(people_NE_inactive_retired_all_list, axis=0, ignore_index=True)



print('Job done, Check the results')



Number of OA areas in iteration:  (1, 8802)
Number of OA areas in iteration:  (2, 8802)
Number of OA areas in iteration:  (3, 8802)
Number of OA areas in iteration:  (4, 8802)
Number of OA areas in iteration:  (5, 8802)
Number of OA areas in iteration:  (6, 8802)
Number of OA areas in iteration:  (7, 8802)
Number of OA areas in iteration:  (8, 8802)
Number of OA areas in iteration:  (9, 8802)
Number of OA areas in iteration:  (10, 8802)
Number of OA areas in iteration:  (11, 8802)
Number of OA areas in iteration:  (12, 8802)
Number of OA areas in iteration:  (13, 8802)
Number of OA areas in iteration:  (14, 8802)
Number of OA areas in iteration:  (15, 8802)
Number of OA areas in iteration:  (16, 8802)
Number of OA areas in iteration:  (17, 8802)
Number of OA areas in iteration:  (18, 8802)
Number of OA areas in iteration:  (19, 8802)
Number of OA areas in iteration:  (20, 8802)
Number of OA areas in iteration:  (21, 8802)
Number of OA areas in iteration:  (22, 8802)
Number of OA areas 

Number of OA areas in iteration:  (182, 8802)
Number of OA areas in iteration:  (183, 8802)
Number of OA areas in iteration:  (184, 8802)
Number of OA areas in iteration:  (185, 8802)
Number of OA areas in iteration:  (186, 8802)
Number of OA areas in iteration:  (187, 8802)
Number of OA areas in iteration:  (188, 8802)
Number of OA areas in iteration:  (189, 8802)
Number of OA areas in iteration:  (190, 8802)
Number of OA areas in iteration:  (191, 8802)
Number of OA areas in iteration:  (192, 8802)
Number of OA areas in iteration:  (193, 8802)
Number of OA areas in iteration:  (194, 8802)
Number of OA areas in iteration:  (195, 8802)
Number of OA areas in iteration:  (196, 8802)
Number of OA areas in iteration:  (197, 8802)
Number of OA areas in iteration:  (198, 8802)
Number of OA areas in iteration:  (199, 8802)
Number of OA areas in iteration:  (200, 8802)
Number of OA areas in iteration:  (201, 8802)
Number of OA areas in iteration:  (202, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (361, 8802)
Number of OA areas in iteration:  (362, 8802)
Number of OA areas in iteration:  (363, 8802)
Number of OA areas in iteration:  (364, 8802)
Number of OA areas in iteration:  (365, 8802)
Number of OA areas in iteration:  (366, 8802)
Number of OA areas in iteration:  (367, 8802)
Number of OA areas in iteration:  (368, 8802)
Number of OA areas in iteration:  (369, 8802)
Number of OA areas in iteration:  (370, 8802)
Number of OA areas in iteration:  (371, 8802)
Number of OA areas in iteration:  (372, 8802)
Number of OA areas in iteration:  (373, 8802)
Number of OA areas in iteration:  (374, 8802)
Number of OA areas in iteration:  (375, 8802)
Number of OA areas in iteration:  (376, 8802)
Number of OA areas in iteration:  (377, 8802)
Number of OA areas in iteration:  (378, 8802)
Number of OA areas in iteration:  (379, 8802)
Number of OA areas in iteration:  (380, 8802)
Number of OA areas in iteration:  (381, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (541, 8802)
Number of OA areas in iteration:  (542, 8802)
Number of OA areas in iteration:  (543, 8802)
Number of OA areas in iteration:  (544, 8802)
Number of OA areas in iteration:  (545, 8802)
Number of OA areas in iteration:  (546, 8802)
Number of OA areas in iteration:  (547, 8802)
Number of OA areas in iteration:  (548, 8802)
Number of OA areas in iteration:  (549, 8802)
Number of OA areas in iteration:  (550, 8802)
Number of OA areas in iteration:  (551, 8802)
Number of OA areas in iteration:  (552, 8802)
Number of OA areas in iteration:  (553, 8802)
Number of OA areas in iteration:  (554, 8802)
Number of OA areas in iteration:  (555, 8802)
Number of OA areas in iteration:  (556, 8802)
Number of OA areas in iteration:  (557, 8802)
Number of OA areas in iteration:  (558, 8802)
Number of OA areas in iteration:  (559, 8802)
Number of OA areas in iteration:  (560, 8802)
Number of OA areas in iteration:  (561, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (720, 8802)
Number of OA areas in iteration:  (721, 8802)
Number of OA areas in iteration:  (722, 8802)
Number of OA areas in iteration:  (723, 8802)
Number of OA areas in iteration:  (724, 8802)
Number of OA areas in iteration:  (725, 8802)
Number of OA areas in iteration:  (726, 8802)
Number of OA areas in iteration:  (727, 8802)
Number of OA areas in iteration:  (728, 8802)
Number of OA areas in iteration:  (729, 8802)
Number of OA areas in iteration:  (730, 8802)
Number of OA areas in iteration:  (731, 8802)
Number of OA areas in iteration:  (732, 8802)
Number of OA areas in iteration:  (733, 8802)
Number of OA areas in iteration:  (734, 8802)
Number of OA areas in iteration:  (735, 8802)
Number of OA areas in iteration:  (736, 8802)
Number of OA areas in iteration:  (737, 8802)
Number of OA areas in iteration:  (738, 8802)
Number of OA areas in iteration:  (739, 8802)
Number of OA areas in iteration:  (740, 8802)
Number of OA areas in iteration:  

...

Number of OA areas in iteration:  (8098, 8802)
Number of OA areas in iteration:  (8099, 8802)
Number of OA areas in iteration:  (8100, 8802)
Number of OA areas in iteration:  (8101, 8802)
Number of OA areas in iteration:  (8102, 8802)
Number of OA areas in iteration:  (8103, 8802)
Number of OA areas in iteration:  (8104, 8802)
Number of OA areas in iteration:  (8105, 8802)
Number of OA areas in iteration:  (8106, 8802)
Number of OA areas in iteration:  (8107, 8802)
Number of OA areas in iteration:  (8108, 8802)
Number of OA areas in iteration:  (8109, 8802)
Number of OA areas in iteration:  (8110, 8802)
Number of OA areas in iteration:  (8111, 8802)
Number of OA areas in iteration:  (8112, 8802)
Number of OA areas in iteration:  (8113, 8802)
Number of OA areas in iteration:  (8114, 8802)
Number of OA areas in iteration:  (8115, 8802)
Number of OA areas in iteration:  (8116, 8802)
Number of OA areas in iteration:  (8117, 8802)
Number of OA areas in iteration:  (8118, 8802)
Number of OA 

Number of OA areas in iteration:  (8274, 8802)
Number of OA areas in iteration:  (8275, 8802)
Number of OA areas in iteration:  (8276, 8802)
Number of OA areas in iteration:  (8277, 8802)
Number of OA areas in iteration:  (8278, 8802)
Number of OA areas in iteration:  (8279, 8802)
Number of OA areas in iteration:  (8280, 8802)
Number of OA areas in iteration:  (8281, 8802)
Number of OA areas in iteration:  (8282, 8802)
Number of OA areas in iteration:  (8283, 8802)
Number of OA areas in iteration:  (8284, 8802)
Number of OA areas in iteration:  (8285, 8802)
Number of OA areas in iteration:  (8286, 8802)
Number of OA areas in iteration:  (8287, 8802)
Number of OA areas in iteration:  (8288, 8802)
Number of OA areas in iteration:  (8289, 8802)
Number of OA areas in iteration:  (8290, 8802)
Number of OA areas in iteration:  (8291, 8802)
Number of OA areas in iteration:  (8292, 8802)
Number of OA areas in iteration:  (8293, 8802)
Number of OA areas in iteration:  (8294, 8802)
Number of OA 

Number of OA areas in iteration:  (8449, 8802)
Number of OA areas in iteration:  (8450, 8802)
Number of OA areas in iteration:  (8451, 8802)
Number of OA areas in iteration:  (8452, 8802)
Number of OA areas in iteration:  (8453, 8802)
Number of OA areas in iteration:  (8454, 8802)
Number of OA areas in iteration:  (8455, 8802)
Number of OA areas in iteration:  (8456, 8802)
Number of OA areas in iteration:  (8457, 8802)
Number of OA areas in iteration:  (8458, 8802)
Number of OA areas in iteration:  (8459, 8802)
Number of OA areas in iteration:  (8460, 8802)
Number of OA areas in iteration:  (8461, 8802)
Number of OA areas in iteration:  (8462, 8802)
Number of OA areas in iteration:  (8463, 8802)
Number of OA areas in iteration:  (8464, 8802)
Number of OA areas in iteration:  (8465, 8802)
Number of OA areas in iteration:  (8466, 8802)
Number of OA areas in iteration:  (8467, 8802)
Number of OA areas in iteration:  (8468, 8802)
Number of OA areas in iteration:  (8469, 8802)
Number of OA 

Number of OA areas in iteration:  (8625, 8802)
Number of OA areas in iteration:  (8626, 8802)
Number of OA areas in iteration:  (8627, 8802)
Number of OA areas in iteration:  (8628, 8802)
Number of OA areas in iteration:  (8629, 8802)
Number of OA areas in iteration:  (8630, 8802)
Number of OA areas in iteration:  (8631, 8802)
Number of OA areas in iteration:  (8632, 8802)
Number of OA areas in iteration:  (8633, 8802)
Number of OA areas in iteration:  (8634, 8802)
Number of OA areas in iteration:  (8635, 8802)
Number of OA areas in iteration:  (8636, 8802)
Number of OA areas in iteration:  (8637, 8802)
Number of OA areas in iteration:  (8638, 8802)
Number of OA areas in iteration:  (8639, 8802)
Number of OA areas in iteration:  (8640, 8802)
Number of OA areas in iteration:  (8641, 8802)
Number of OA areas in iteration:  (8642, 8802)
Number of OA areas in iteration:  (8643, 8802)
Number of OA areas in iteration:  (8644, 8802)
Number of OA areas in iteration:  (8645, 8802)
Number of OA 

Number of OA areas in iteration:  (8800, 8802)
Number of OA areas in iteration:  (8801, 8802)
Number of OA areas in iteration:  (8802, 8802)
Job done, Check the results


In [36]:
# Check the amount of people selected as retired:
len(df_people_NE_inactive_retired_all_v2)

56289

In [37]:
#####################################################
# df_people_NE_inactive_retired_all_v2
##   CHECKS:
    
# Calculate the % of students within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_students = len(df_people_NE_inactive_retired_all_v2)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_retired was', percentage_students)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_lookingafter_male = df_people_NE_inactive_retired_all_v2.loc[(df_people_NE_inactive_retired_all_v2['Sex'] == 1)]
percentage_males_students = len(df_persons_NE_inactive_lookingafter_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_retired was', percentage_males_students)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_lookingafter_female = df_people_NE_inactive_retired_all_v2.loc[(df_people_NE_inactive_retired_all_v2['Sex'] == 2)]
percentage_females_students = len(df_persons_NE_inactive_lookingafter_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_retired was', percentage_females_students)
    

# Calculate the % proportion of males that are students
percentage_proportion_male = len(df_persons_NE_inactive_lookingafter_male)/(len(df_people_NE_inactive_retired_all_v2)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are students
percentage_proportion_female = len(df_persons_NE_inactive_lookingafter_female)/(len(df_people_NE_inactive_retired_all_v2)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


Value percentage_retired was 13.804071402821682
Value percentage_males_retired was 17.9360815959486
Value percentage_females_retired was 10.87864356232434
Value percentage_proportion_male was 53.85954627014159
Value percentage_proportion_female was 46.14045372985841
Code finished. Check the results!


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_retired_conversor" parameter value.

In [38]:
# Check if there are duplicates:
df_people_NE_inactive_retired_all_v2_1 = df_people_NE_inactive_retired_all_v2

# Remove duplicates
df_people_NE_inactive_retired_all_v2_1 = df_people_NE_inactive_retired_all_v2_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_people_NE_inactive_retired_all_v2_1)

56289

In [39]:
# Save dataframe
df_people_NE_inactive_retired_all_v2_export_20220302_correct = df_people_NE_inactive_retired_all_v2
df_people_NE_inactive_retired_all_v2_export_20220302_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_people_NE_inactive_retired_all_v2_export_20220302_correct.csv', encoding='utf-8', header=True)


## Sick

In [40]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_plus_retired = (pd.concat([df_persons_NE_inactive_16_64_NO_students_NO_lookingafter, df_people_NE_inactive_retired_all_v2]))

# Remove duplicates and keep only those who were not assigned a driving licence
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_plus_retired.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired)

161865

In [41]:
# Check the number of remaining people to be assigned as "Retired", "Sick" or "Other"

len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3) - len(df_people_NE_inactive_retired_all_v2)

161865

In [42]:
# Calculate the differences between 
len(df_persons_NE_inactive_16_64) - len(df_inactive_students_all) - len(df_inactive_lookingafter_all_22feb_3) - len(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired) - len(df_people_NE_inactive_retired_all_v2)

0

In [57]:
# Start developing the code to identify LONGTERM SICK people

## LIST OF OA_AREAS that has been generated before
# Create a list with all Households unique ID values
AreaOA_list = df_households_NE_extended['Area_OA'].tolist()
# Remove duplicates
AreaOA_list =  list(set(AreaOA_list))


# List for the gender types: male (1) and female (2)
gender_list = [1,2]

#Create an empty list where the small blocks of dataframes of "LONGTERM SICK" people will be stored
people_NE_inactive_longtermsick_all_list = []


# % inactive conversor value 2011 to 2019
## This value is converting the % of inactive people (each type) from 2011 to 2019 based on data from "Regional labour market statistics: HI01 Headline indicators for the North East"
inactive_longtermsick_conversor = 3.0


#  Relationship between male and female people per reason of inactivity (male + female =1)
## This values are coming from Regional labour market statistics:HI01 Headline indicators for the North East. Tables 10b and 10c
### See Excel "Economic_activity_INACTIVE_types.xlsx"
#### "C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\Validation_results\Economic_activity_INACTIVE_types.xlsx"
# Males
percentage_1_longtermsick = 0.584

# Females
percentage_2_longtermsick = 0.416




#Create a variable that counts the number of OA areas iterated
OA_area_counter = 0

for OA_area in AreaOA_list:

    OA_area_counter += 1
    print("Number of OA areas in iteration: ", (OA_area_counter, len(AreaOA_list)))
    
    # Select the row of the df_QS601EW_inactive that is related to the selected OA area:
    df_QS601EW_inactive_OAarea = df_QS601EW_inactive.loc[(df_QS601EW_inactive['geography'] == OA_area)]
    
    # Get the VALUE of the total number of people living in the seleted OA area that are inactive from table QS601EW
    total_QS601EW_inactive_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Total')]
    
    # Select those people that live in the selected OA area (people already between 16 and 64 years old):
    df_persons_NE_inactive_16_64_OAarea = df_persons_NE_inactive_16_64.loc[(df_persons_NE_inactive_16_64['Area_OA_x'] == OA_area)]
    
    # Calculate the total number of people (aged 16-64) that live in the selected OA_area from SPENSER data
    total_inactive_in_OA_2019 = len(df_persons_NE_inactive_16_64_OAarea)

    
    
    # Calculate the ratio of people 2019 vs 2011 (to increase or decrease the number of people that have to be selected per type of inactivity (retired, student, looking after, disable or other))
    if total_QS601EW_inactive_in_OA_2011 > 0:
        ratio_people_2019_2011 = total_inactive_in_OA_2019/total_QS601EW_inactive_in_OA_2011
        #print('Ratio people 2019_2011')
        #print(ratio_people_2019_2011)
    else:
        ratio_people_2019_2011 = 1
    
    
    # Get the VALUE of the number of people living in the seleted OA area that are inactive LONGTERMSICK from table QS601EW
    total_QS601EW_inactive_longtermsick_in_OA_2011 = df_QS601EW_inactive_OAarea.iloc[0,df_QS601EW_inactive_OAarea.columns.get_loc('Economically_inactive_Longterm_sick_or_disabled')]


    # ASSIGNMENT OF INACTIVE reasons based on:
    # 1. the value from 2011 (table QS601EW) related to each category (student, looking after, retired...)
    # 2. the inactive_XXXX_conversor
    # 3. the ratio of people 2019 vs 2011 ("ratio_people_2019_2011") [CONSTANT VALUE FOR ALL CATEGORIES]
    

    # New value for 2019 =  (Value from 2011 (table QS601EW)) * ("inactive_longtermsick_conversor"  value) * (ratio of people 2019 vs 2011 ("ratio_people_2019_2011")) 
    ## This value is the number of people that will be assigned "LONG TERM SICK" in the OA area selected
    inactive_longtermsick_2019 = int(round(total_QS601EW_inactive_longtermsick_in_OA_2011 * inactive_longtermsick_conversor * ratio_people_2019_2011,0)) 

    

    #################################################################################################################
    ## START IDENTIFYING THE "LONGTERM SICK" ATTRIBUTE VALUE
    
    
    # SELECT people as "LONGTERM SICK" BASED ON THEIR SOCIOECONOMIC CHARACTERISTICS:
    ## First selection:
    ### 1. People aged 50-64
    ### 2. Sex: proportions based on variables "percentage_1_longtermsick" and "percentage_2_longtermsick" assigned previously
    # In case there are still leftovers:
    ## Second selection:
    ### 1. People aged < 50
    ### 2. Sex: proportions based on variables "percentage_1_longtermsick" and "percentage_2_longtermsick" assigned previously

    for gender in gender_list:
        
        # Dataframe for First selection
        ## people of a specific sex, age >=50 and specific OA area
        df_people_inactive_longtermsick_50_64_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Age'] >= 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Area_OA_x'] == OA_area)]
        
        # Dataframe for Second selection
        ## people of a specific sex, age >50 and specific OA area
        df_people_inactive_longtermsick_50less_gender = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired.loc[(df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Age'] < 50) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Sex'] == gender) & (df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired['Area_OA_x'] == OA_area)]
        

        if (len(df_people_inactive_longtermsick_50_64_gender) > int(round(inactive_longtermsick_2019 * globals()[f"percentage_{gender}_longtermsick"],0))):
        
            ## First selection:
            df_first_selection_longtermsick = df_people_inactive_longtermsick_50_64_gender.sample(int(round(inactive_longtermsick_2019 * globals()[f"percentage_{gender}_longtermsick"],0)))

            df_second_selection_longtermsick = pd.DataFrame()
            
        elif (len(df_people_inactive_longtermsick_50_64_gender) == int(round(inactive_longtermsick_2019 * globals()[f"percentage_{gender}_longtermsick"],0))):
            
            ## First selection:
            df_first_selection_longtermsick = df_people_inactive_longtermsick_50_64_gender

            df_second_selection_longtermsick = pd.DataFrame()
            
            
        else:
            ## First selection:
            df_first_selection_longtermsick = df_people_inactive_longtermsick_50_64_gender

            # There are still remaining some people to be assigned as "LONGTERM SICK"
            if (len(df_people_inactive_longtermsick_50less_gender) > int(round(inactive_longtermsick_2019 * globals()[f"percentage_{gender}_longtermsick"],0) - len(df_first_selection_longtermsick))):

                ## Second selection:
                df_second_selection_longtermsick = df_people_inactive_longtermsick_50less_gender.sample(int(round(inactive_longtermsick_2019 * globals()[f"percentage_{gender}_longtermsick"],0)) - len(df_first_selection_longtermsick))

            else:
                
                ## Second selection:
                df_second_selection_longtermsick = df_people_inactive_longtermsick_50less_gender

    
        # Once all possible ways to select people have been computed, there is a need to concatenate all dataframes:
        df_people_NE_inactive_longtermsick = (pd.concat([df_first_selection_longtermsick, df_second_selection_longtermsick]))
 
        #Append the dataframe into the temporal list
        people_NE_inactive_longtermsick_all_list.append(df_people_NE_inactive_longtermsick)
        

#concatenate all people "LONGTERM SICK" in one dataframe
df_people_NE_inactive_longtermsick_all = pd.concat(people_NE_inactive_longtermsick_all_list, axis=0, ignore_index=True)



print('Job done, Check the results')

Number of OA areas in iteration:  (1, 8802)
Number of OA areas in iteration:  (2, 8802)
Number of OA areas in iteration:  (3, 8802)
Number of OA areas in iteration:  (4, 8802)
Number of OA areas in iteration:  (5, 8802)
Number of OA areas in iteration:  (6, 8802)
Number of OA areas in iteration:  (7, 8802)
Number of OA areas in iteration:  (8, 8802)
Number of OA areas in iteration:  (9, 8802)
Number of OA areas in iteration:  (10, 8802)
Number of OA areas in iteration:  (11, 8802)
Number of OA areas in iteration:  (12, 8802)
Number of OA areas in iteration:  (13, 8802)
Number of OA areas in iteration:  (14, 8802)
Number of OA areas in iteration:  (15, 8802)
Number of OA areas in iteration:  (16, 8802)
Number of OA areas in iteration:  (17, 8802)
Number of OA areas in iteration:  (18, 8802)
Number of OA areas in iteration:  (19, 8802)
Number of OA areas in iteration:  (20, 8802)
Number of OA areas in iteration:  (21, 8802)
Number of OA areas in iteration:  (22, 8802)
Number of OA areas 

Number of OA areas in iteration:  (182, 8802)
Number of OA areas in iteration:  (183, 8802)
Number of OA areas in iteration:  (184, 8802)
Number of OA areas in iteration:  (185, 8802)
Number of OA areas in iteration:  (186, 8802)
Number of OA areas in iteration:  (187, 8802)
Number of OA areas in iteration:  (188, 8802)
Number of OA areas in iteration:  (189, 8802)
Number of OA areas in iteration:  (190, 8802)
Number of OA areas in iteration:  (191, 8802)
Number of OA areas in iteration:  (192, 8802)
Number of OA areas in iteration:  (193, 8802)
Number of OA areas in iteration:  (194, 8802)
Number of OA areas in iteration:  (195, 8802)
Number of OA areas in iteration:  (196, 8802)
Number of OA areas in iteration:  (197, 8802)
Number of OA areas in iteration:  (198, 8802)
Number of OA areas in iteration:  (199, 8802)
Number of OA areas in iteration:  (200, 8802)
Number of OA areas in iteration:  (201, 8802)
Number of OA areas in iteration:  (202, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (362, 8802)
Number of OA areas in iteration:  (363, 8802)
Number of OA areas in iteration:  (364, 8802)
Number of OA areas in iteration:  (365, 8802)
Number of OA areas in iteration:  (366, 8802)
Number of OA areas in iteration:  (367, 8802)
Number of OA areas in iteration:  (368, 8802)
Number of OA areas in iteration:  (369, 8802)
Number of OA areas in iteration:  (370, 8802)
Number of OA areas in iteration:  (371, 8802)
Number of OA areas in iteration:  (372, 8802)
Number of OA areas in iteration:  (373, 8802)
Number of OA areas in iteration:  (374, 8802)
Number of OA areas in iteration:  (375, 8802)
Number of OA areas in iteration:  (376, 8802)
Number of OA areas in iteration:  (377, 8802)
Number of OA areas in iteration:  (378, 8802)
Number of OA areas in iteration:  (379, 8802)
Number of OA areas in iteration:  (380, 8802)
Number of OA areas in iteration:  (381, 8802)
Number of OA areas in iteration:  (382, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (542, 8802)
Number of OA areas in iteration:  (543, 8802)
Number of OA areas in iteration:  (544, 8802)
Number of OA areas in iteration:  (545, 8802)
Number of OA areas in iteration:  (546, 8802)
Number of OA areas in iteration:  (547, 8802)
Number of OA areas in iteration:  (548, 8802)
Number of OA areas in iteration:  (549, 8802)
Number of OA areas in iteration:  (550, 8802)
Number of OA areas in iteration:  (551, 8802)
Number of OA areas in iteration:  (552, 8802)
Number of OA areas in iteration:  (553, 8802)
Number of OA areas in iteration:  (554, 8802)
Number of OA areas in iteration:  (555, 8802)
Number of OA areas in iteration:  (556, 8802)
Number of OA areas in iteration:  (557, 8802)
Number of OA areas in iteration:  (558, 8802)
Number of OA areas in iteration:  (559, 8802)
Number of OA areas in iteration:  (560, 8802)
Number of OA areas in iteration:  (561, 8802)
Number of OA areas in iteration:  (562, 8802)
Number of OA areas in iteration:  

Number of OA areas in iteration:  (723, 8802)
Number of OA areas in iteration:  (724, 8802)
Number of OA areas in iteration:  (725, 8802)
Number of OA areas in iteration:  (726, 8802)
Number of OA areas in iteration:  (727, 8802)
Number of OA areas in iteration:  (728, 8802)
Number of OA areas in iteration:  (729, 8802)
Number of OA areas in iteration:  (730, 8802)
Number of OA areas in iteration:  (731, 8802)
Number of OA areas in iteration:  (732, 8802)
Number of OA areas in iteration:  (733, 8802)
Number of OA areas in iteration:  (734, 8802)
Number of OA areas in iteration:  (735, 8802)
Number of OA areas in iteration:  (736, 8802)
Number of OA areas in iteration:  (737, 8802)
Number of OA areas in iteration:  (738, 8802)
Number of OA areas in iteration:  (739, 8802)
Number of OA areas in iteration:  (740, 8802)
Number of OA areas in iteration:  (741, 8802)
Number of OA areas in iteration:  (742, 8802)
Number of OA areas in iteration:  (743, 8802)
Number of OA areas in iteration:  

...

Number of OA areas in iteration:  (7944, 8802)
Number of OA areas in iteration:  (7945, 8802)
Number of OA areas in iteration:  (7946, 8802)
Number of OA areas in iteration:  (7947, 8802)
Number of OA areas in iteration:  (7948, 8802)
Number of OA areas in iteration:  (7949, 8802)
Number of OA areas in iteration:  (7950, 8802)
Number of OA areas in iteration:  (7951, 8802)
Number of OA areas in iteration:  (7952, 8802)
Number of OA areas in iteration:  (7953, 8802)
Number of OA areas in iteration:  (7954, 8802)
Number of OA areas in iteration:  (7955, 8802)
Number of OA areas in iteration:  (7956, 8802)
Number of OA areas in iteration:  (7957, 8802)
Number of OA areas in iteration:  (7958, 8802)
Number of OA areas in iteration:  (7959, 8802)
Number of OA areas in iteration:  (7960, 8802)
Number of OA areas in iteration:  (7961, 8802)
Number of OA areas in iteration:  (7962, 8802)
Number of OA areas in iteration:  (7963, 8802)
Number of OA areas in iteration:  (7964, 8802)
Number of OA 

Number of OA areas in iteration:  (8121, 8802)
Number of OA areas in iteration:  (8122, 8802)
Number of OA areas in iteration:  (8123, 8802)
Number of OA areas in iteration:  (8124, 8802)
Number of OA areas in iteration:  (8125, 8802)
Number of OA areas in iteration:  (8126, 8802)
Number of OA areas in iteration:  (8127, 8802)
Number of OA areas in iteration:  (8128, 8802)
Number of OA areas in iteration:  (8129, 8802)
Number of OA areas in iteration:  (8130, 8802)
Number of OA areas in iteration:  (8131, 8802)
Number of OA areas in iteration:  (8132, 8802)
Number of OA areas in iteration:  (8133, 8802)
Number of OA areas in iteration:  (8134, 8802)
Number of OA areas in iteration:  (8135, 8802)
Number of OA areas in iteration:  (8136, 8802)
Number of OA areas in iteration:  (8137, 8802)
Number of OA areas in iteration:  (8138, 8802)
Number of OA areas in iteration:  (8139, 8802)
Number of OA areas in iteration:  (8140, 8802)
Number of OA areas in iteration:  (8141, 8802)
Number of OA 

Number of OA areas in iteration:  (8297, 8802)
Number of OA areas in iteration:  (8298, 8802)
Number of OA areas in iteration:  (8299, 8802)
Number of OA areas in iteration:  (8300, 8802)
Number of OA areas in iteration:  (8301, 8802)
Number of OA areas in iteration:  (8302, 8802)
Number of OA areas in iteration:  (8303, 8802)
Number of OA areas in iteration:  (8304, 8802)
Number of OA areas in iteration:  (8305, 8802)
Number of OA areas in iteration:  (8306, 8802)
Number of OA areas in iteration:  (8307, 8802)
Number of OA areas in iteration:  (8308, 8802)
Number of OA areas in iteration:  (8309, 8802)
Number of OA areas in iteration:  (8310, 8802)
Number of OA areas in iteration:  (8311, 8802)
Number of OA areas in iteration:  (8312, 8802)
Number of OA areas in iteration:  (8313, 8802)
Number of OA areas in iteration:  (8314, 8802)
Number of OA areas in iteration:  (8315, 8802)
Number of OA areas in iteration:  (8316, 8802)
Number of OA areas in iteration:  (8317, 8802)
Number of OA 

Number of OA areas in iteration:  (8473, 8802)
Number of OA areas in iteration:  (8474, 8802)
Number of OA areas in iteration:  (8475, 8802)
Number of OA areas in iteration:  (8476, 8802)
Number of OA areas in iteration:  (8477, 8802)
Number of OA areas in iteration:  (8478, 8802)
Number of OA areas in iteration:  (8479, 8802)
Number of OA areas in iteration:  (8480, 8802)
Number of OA areas in iteration:  (8481, 8802)
Number of OA areas in iteration:  (8482, 8802)
Number of OA areas in iteration:  (8483, 8802)
Number of OA areas in iteration:  (8484, 8802)
Number of OA areas in iteration:  (8485, 8802)
Number of OA areas in iteration:  (8486, 8802)
Number of OA areas in iteration:  (8487, 8802)
Number of OA areas in iteration:  (8488, 8802)
Number of OA areas in iteration:  (8489, 8802)
Number of OA areas in iteration:  (8490, 8802)
Number of OA areas in iteration:  (8491, 8802)
Number of OA areas in iteration:  (8492, 8802)
Number of OA areas in iteration:  (8493, 8802)
Number of OA 

Number of OA areas in iteration:  (8649, 8802)
Number of OA areas in iteration:  (8650, 8802)
Number of OA areas in iteration:  (8651, 8802)
Number of OA areas in iteration:  (8652, 8802)
Number of OA areas in iteration:  (8653, 8802)
Number of OA areas in iteration:  (8654, 8802)
Number of OA areas in iteration:  (8655, 8802)
Number of OA areas in iteration:  (8656, 8802)
Number of OA areas in iteration:  (8657, 8802)
Number of OA areas in iteration:  (8658, 8802)
Number of OA areas in iteration:  (8659, 8802)
Number of OA areas in iteration:  (8660, 8802)
Number of OA areas in iteration:  (8661, 8802)
Number of OA areas in iteration:  (8662, 8802)
Number of OA areas in iteration:  (8663, 8802)
Number of OA areas in iteration:  (8664, 8802)
Number of OA areas in iteration:  (8665, 8802)
Number of OA areas in iteration:  (8666, 8802)
Number of OA areas in iteration:  (8667, 8802)
Number of OA areas in iteration:  (8668, 8802)
Number of OA areas in iteration:  (8669, 8802)
Number of OA 

In [58]:
# Check the number of people selected as inactive long time sick:
len(df_people_NE_inactive_longtermsick_all)

123871

In [59]:
# Check if there are duplicates:
df_people_NE_inactive_longtermsick_all_1 = df_people_NE_inactive_longtermsick_all

# Remove duplicates
df_people_NE_inactive_longtermsick_all_1 = df_people_NE_inactive_longtermsick_all_1.drop_duplicates(subset='PID_AreaMSOA',keep = False)


len(df_people_NE_inactive_longtermsick_all_1)

123871

In [60]:
#####################################################
# df_people_NE_inactive_retired_all_v2
##   CHECKS:
    
# Calculate the % of students within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_sick = len(df_people_NE_inactive_longtermsick_all)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_sick was', percentage_sick)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_sick_male = df_people_NE_inactive_longtermsick_all.loc[(df_people_NE_inactive_longtermsick_all['Sex'] == 1)]
percentage_males_sick = len(df_persons_NE_inactive_sick_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_students was', percentage_males_sick)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_sick_female = df_people_NE_inactive_longtermsick_all.loc[(df_people_NE_inactive_longtermsick_all['Sex'] == 2)]
percentage_females_sick = len(df_persons_NE_inactive_sick_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_sick was', percentage_females_sick)
    

# Calculate the % proportion of males that are students
percentage_proportion_male = len(df_persons_NE_inactive_sick_male)/(len(df_people_NE_inactive_longtermsick_all)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are students
percentage_proportion_female = len(df_persons_NE_inactive_sick_female)/(len(df_people_NE_inactive_longtermsick_all)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


Value percentage_sick was 30.377589382275836
Value percentage_males_students was 34.33750621198855
Value percentage_females_sick was 27.57400216969712
Value percentage_proportion_male was 46.855196131459344
Value percentage_proportion_female was 53.144803868540656
Code finished. Check the results!


##### If the results obtained are not close enough to those provided by the "Regional labour market statistics: HI01 Headline indicators for the North East (Tables 10a, 10b, 10c)", then
- change the "inactive_longtermsick_conversor" parameter value.

In [61]:
# Save dataframe
df_people_NE_inactive_longtermsick_all_export_20220302_correct = df_people_NE_inactive_longtermsick_all
df_people_NE_inactive_longtermsick_all_export_20220302_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_people_NE_inactive_longtermsick_all_export_20220302_correct.csv', encoding='utf-8', header=True)


## Other

In [62]:
# KEEP ONLY THOSE INACTIVE THAT ARE NOT STUDENTS
# Concatenate the dataframe of all inactive people (16-64) + the ones selected before as STUDENTS
df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired_plus_sick = (pd.concat([df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired, df_people_NE_inactive_longtermsick_all]))

# Remove duplicates and keep only those who were not assigned a driving licence
df_persons_NE_inactive_16_64_Other = df_persons_NE_inactive_16_64_NO_students_NO_lookingafter_NO_retired_plus_sick.drop_duplicates(subset='PID_AreaMSOA', keep = False)

len(df_persons_NE_inactive_16_64_Other)

37994

In [65]:
##   CHECKS:
    
# Calculate the % of students within the inactive people
df_persons_inactive_16_64_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64)]
    
percentage_other = len(df_persons_NE_inactive_16_64_Other)/ len(df_persons_inactive_16_64_check) *100
print('Value percentage_sick was', percentage_other)
    
    
# males 16-64 that are inactive:
df_persons_inactive_16_64_MALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 1)] 
    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_Other_male = df_persons_NE_inactive_16_64_Other.loc[(df_persons_NE_inactive_16_64_Other['Sex'] == 1)]
percentage_males_other = len(df_persons_NE_inactive_Other_male)/ len(df_persons_inactive_16_64_MALE_check) *100
print('Value percentage_males_students was', percentage_males_other)
    
    
# females 16-64 that are inactive:
df_persons_inactive_16_64_FEMALE_check = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >=16) & (df_persons_NE_inactive_20211117['Age'] <=64) & (df_persons_NE_inactive_20211117['Sex'] == 2)]

    
# Calculat the % of inactive men that are students
df_persons_NE_inactive_Other_female = df_persons_NE_inactive_16_64_Other.loc[(df_persons_NE_inactive_16_64_Other['Sex'] == 2)]
percentage_females_other = len(df_persons_NE_inactive_Other_female)/ len(df_persons_inactive_16_64_FEMALE_check) *100
print('Value percentage_females_other was', percentage_females_other)
    

# Calculate the % proportion of males that are students
percentage_proportion_male = len(df_persons_NE_inactive_Other_male)/(len(df_persons_NE_inactive_16_64_Other)) * 100
print('Value percentage_proportion_male was', percentage_proportion_male)

# Calculate the % proportion of females that are students
percentage_proportion_female = len(df_persons_NE_inactive_Other_female)/(len(df_persons_NE_inactive_16_64_Other)) * 100
print('Value percentage_proportion_female was', percentage_proportion_female)

print('Code finished. Check the results!')


Value percentage_sick was 9.3174845685446
Value percentage_males_students was 3.316610265754786
Value percentage_females_other was 13.56605219838906
Value percentage_proportion_male was 14.754961309680475
Value percentage_proportion_female was 85.24503869031952
Code finished. Check the results!


In [66]:
# Save dataframe
df_persons_NE_inactive_16_64_Other_all_export_20220302_correct = df_persons_NE_inactive_16_64_Other
df_persons_NE_inactive_16_64_Other_all_export_20220302_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_persons_NE_inactive_16_64_Other_all_export_20220302_correct.csv', encoding='utf-8', header=True)


In [69]:
# Check the total number of people selected as student, looking after, retired, sick or other
a = len(df_inactive_students_all) + len(df_inactive_lookingafter_all_22feb_3) + len(df_people_NE_inactive_retired_all_v2) + len(df_people_NE_inactive_longtermsick_all) + len(df_persons_NE_inactive_16_64_Other)

In [70]:
# Check the total number to be assigned an inactive category
b = len(df_persons_NE_inactive_16_64)

In [71]:
# Number of people that were not assigned an inactive category:
a-b

0

In [ ]:
####################################################################################

In [7]:
# Create a dataframe containing only the people that are Economic_activity = 'Inactive' and age 65 or more
## All these people will be considered as RETIRED ("Occupation" attribute value will be RETIRED)
df_persons_NE_inactive_65_120 = df_persons_NE_inactive_20211117.loc[(df_persons_NE_inactive_20211117['Age'] >= 65)]


In [8]:
len(df_persons_NE_inactive_65_120)

505273

In [ ]:
# Save dataframe
df_persons_NE_inactive_65_120_export_20220304_correct = df_persons_NE_inactive_65_120
df_persons_NE_inactive_65_120_export_20220304_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_persons_NE_inactive_65_120_export_20220304_correct.csv', encoding='utf-8', header=True)


### Update the 'Occupation' column value in each of the sub-dataframes generated

In [ ]:
df_inactive_students_all['Occupation'] = 'Student'

In [ ]:
df_inactive_lookingafter_all_22feb_3['Occupation'] = 'Looking after home family'

In [ ]:
df_people_NE_inactive_retired_all_v2['Occupation'] = 'Retired'

In [ ]:
df_people_NE_inactive_longtermsick_all['Occupation'] = 'Long term sick'

In [ ]:
df_persons_NE_inactive_16_64_Other['Occupation'] = 'Other'

In [ ]:
df_persons_NE_inactive_65_120['Occupation'] = 'Retired'

### concatenate all dataframe in a single dataframe

In [ ]:
df_persons_NE_inactive_5categories = (pd.concat([df_inactive_students_all, df_inactive_lookingafter_all_22feb_3, df_people_NE_inactive_retired_all_v2, df_people_NE_inactive_longtermsick_all, df_persons_NE_inactive_16_64_Other, df_persons_NE_inactive_65_120]))


In [ ]:
# Save dataframe
df_persons_NE_inactive_5categories_export_20220304_correct = df_persons_NE_inactive_5categories
df_persons_NE_inactive_5categories_export_20220304_correct.to_csv(r'C:\Users\b9055315\Documents\PhD_PROJECT\Synthetic_population_developement\SPENSER\Data\2019\economic_activity\Inactive_categories\df_persons_NE_inactive_5categories_export_20220304_correct.csv', encoding='utf-8', header=True)
